# Spelplatser

Vad finns det att säga om var matcherna har spelats?

Jag har samlat in koordinaterna för alla allsvenska lags hemmaarenor för att kunna titta på saker som vilken säsong som var nordligast, och var varje säsong spelats i genomsnitt.

*OBS:* En del lag har bytt arena under årens gång. Det har jag inte tagit med i datan än så länge.

In [4]:
%matplotlib inline
import requests
import json
import numpy as np
import pandas as pd
import scipy.stats as stats
import matplotlib.pyplot as plt
from matplotlib import colormaps as cmaps

# plt.rcParams['text.usetex'] = True
plt.rcParams['font.family'] = 'serif'

In [7]:
df = pd.read_csv('../data/allsvenskan.csv')
df['ppg_3pwin'] = (df.wins*3 + df.draws)/df.games
venues = pd.read_csv('../data/venues.csv')
# final_rounds = df[df['final'] == True]
final_rounds = df.query('final == 1')
since_2008 = df.loc[2008:]


In [8]:
import plotly.express as px

df_venues = final_rounds.merge(venues, on='team_id')
mean_coords = df_venues.pivot_table(values=['venue_lat', 'venue_lon'], index='year')
mean_coords

wh = 1080

center = dict(lat=mean_coords.venue_lat.mean()+.2, lon=mean_coords.venue_lon.mean())
fig = px.scatter_mapbox(mean_coords, lat='venue_lat', lon='venue_lon', size='venue_lat', size_max=10, color=mean_coords.index, opacity=.7, center=center, color_continuous_scale='bluered', zoom=6.5, width=wh, height=wh)
fig.update_layout(mapbox_style='carto-positron', paper_bgcolor='#D6D7D7', title=dict(text='Genomsnittlig spelplats, Allsvenskan 1924-2023', font=dict(family='serif', size=20)), margin=dict(autoexpand=True, l=25, t=50, r=25, b=25), font=dict(family='serif'))
fig.show()
